In [63]:
import pandas as pd
import warnings

In [64]:
df = pd.read_csv("prideNprejudice.csv")

In [65]:
df.head()

,sentences
0,"It is a truth universally acknowledged, that a..."
1,"""My dear Mr. Bennet,"" said his lady to him one..."
2,Bennet replied that he had not.
3,"""But it is,"" returned she; ""for Mrs. Long has ..."
4,Bennet made no answer.


In [66]:
size = len(df)

In [67]:
import bs4 as bs
import nltk
from nltk.tokenize import sent_tokenize # tokenizes sentences
import re
from nltk.corpus import stopwords

In [68]:
#From NLTK notebook shown
def sentence_cleaner(sentence):
    '''
    Clean and preprocess a review.
    
    1. Remove HTML tags
    2. Use regex to remove all special characters (only keep letters)
    3. Make strings to lower case and tokenize / word split reviews
    4. Remove English stopwords
    5. Rejoin to one string
    '''
    
    #1. Remove HTML tags
    sentence = bs.BeautifulSoup(sentence).text
    warnings.filterwarnings('ignore')
    #2. Use regex to find emoticons
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', sentence)
    
    #3. Remove punctuation
    sentence = re.sub("[^a-zA-Z]", " ",sentence)
    
    #4. Tokenize into words (all lower case)
    sentence = sentence.lower().split()
    
    #5. Remove stopwords
    eng_stopwords = set(stopwords.words("english"))
    sentence = [w for w in sentence if not w in eng_stopwords]
    
    #6. Join the review to one sentence
    sentence = ' '.join(sentence+emoticons)
    # add emoticons to the end

    return(sentence)

In [69]:
%%time

sentence_clean_original = []

for i in range(0,size):
    sentence_clean_original.append(sentence_cleaner(df['sentences'][i]))

Wall time: 19.3 s


In [70]:
sentence_clean_original[0]

'truth universally acknowledged single man possessionof good fortune must want wife however little known feelings views man may hisfirst entering neighbourhood truth well fixed mindsof surrounding families considered rightful propertyof one daughters'

In [71]:
l = []
for i in range (0,len(sentence_clean_original)):
    mystr = sentence_clean_original[i]
    wordList = re.sub("[^\w]", " ",  mystr).split()
    l = l + wordList

In [72]:
l[0]

'truth'

In [73]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [74]:
model = gensim.models.Word2Vec([l], size=len(l))
model.train([l],total_examples=len(l),epochs=10)

2018-04-24 23:53:21,047 : INFO : collecting all words and their counts
2018-04-24 23:53:21,072 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-24 23:53:21,115 : INFO : collected 11632 word types from a corpus of 55184 raw words and 1 sentences
2018-04-24 23:53:21,121 : INFO : Loading a fresh vocabulary
2018-04-24 23:53:21,146 : INFO : min_count=5 retains 1817 unique words (15% of original 11632, drops 9815)
2018-04-24 23:53:21,149 : INFO : min_count=5 leaves 41518 word corpus (75% of original 55184, drops 13666)
2018-04-24 23:53:21,278 : INFO : deleting the raw counts dictionary of 11632 items
2018-04-24 23:53:21,283 : INFO : sample=0.001 downsamples 53 most-common words
2018-04-24 23:53:21,287 : INFO : downsampling leaves estimated 37407 word corpus (90.1% of prior 41518)
2018-04-24 23:53:21,328 : INFO : estimated required memory for 1817 words and 55184 dimensions: 803063124 bytes
2018-04-24 23:53:21,331 : INFO : resetting layer weights
2018-04-24 2

2018-04-24 23:57:10,272 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-24 23:57:10,276 : INFO : EPOCH - 7 : training on 55184 raw words (10000 effective words) took 13.6s, 736 effective words/s
2018-04-24 23:57:10,281 : WARNING : EPOCH - 7 : supplied example count (1) did not equal expected count (55184)
2018-04-24 23:57:10,299 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-24 23:57:10,305 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-24 23:57:24,443 : INFO : EPOCH 8 - PROGRESS: at 0.00% examples, 707 words/s, in_qsize 0, out_qsize 1
2018-04-24 23:57:24,445 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-24 23:57:24,449 : INFO : EPOCH - 8 : training on 55184 raw words (10000 effective words) took 14.2s, 706 effective words/s
2018-04-24 23:57:24,453 : WARNING : EPOCH - 8 : supplied example count (1) did not equal expected count (55184)
2018-04-24 23:57:24,472 : INFO : worker t

(100000, 551840)

In [75]:
print(model['girls'])

[ 0.0089632  -0.0092427  -0.03871374 ..., -0.01692482  0.00929901
 -0.00245467]


In [76]:
model.wv.similarity('elizabeth','girl')

0.99999937437168585

In [80]:
model.wv.most_similar(["girl"],topn = 6)

2018-04-24 23:59:47,374 : INFO : precomputing L2-norms of word weight vectors


[('rest', 1.0000008344650269),
 ('making', 1.0000007152557373),
 ('consequence', 1.0000004768371582),
 ('temper', 1.0000003576278687),
 ('avoid', 1.000000238418579),
 ('life', 1.000000238418579)]

In [87]:
model.wv.doesnt_match(["girls","elizabeth","universally","lady"])

2018-04-25 00:03:10,486 : WARNING : vectors for words {'universally'} are not present in the model, ignoring these words


'girls'

In [88]:
model.wv.similarity('bennet','long')

0.99999985882839926